In [1]:
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
import os.path as osp
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
from PIL import Image, ImageFilter

In [2]:
from tqdm import tqdm
import shutil
from multiprocessing import Pool
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from byol_pytorch import BYOL
from torchvision import models
import torchvision.transforms as T
import random
from torch import nn
import time
import scanpy as sc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

<h4>Input</h4>

In [8]:
dataset = '151674'
file_fold = '/home/isaac/dev/sfu/cmpt415/DeepST/data/DLPFC/' + str(dataset)
adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True)
adata.var_names_make_unique()
adata

/tmp/ipykernel_6316/2328745661.py:3: FutureWarning: Use `squidpy.read.visium` instead.
  adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True)
/home/isaac/dev/sfu/cmpt415/CLGraph/CLGraph/clgraph/lib/python3.12/site-packages/anndata/_core/anndata.py:1776: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/isaac/dev/sfu/cmpt415/CLGraph/CLGraph/clgraph/lib/python3.12/site-packages/anndata/_core/anndata.py:1776: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 3673 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [17]:
im_path = "/home/isaac/dev/sfu/cmpt415/hest_data/wsis/MISC3.tif"
im = cv2.imread(im_path,1)   
im.shape

(13332, 13332, 3)

<h4>Clipping</h4>

In [11]:
# Create directory for clipped images
clip_image_path = os.path.join(file_fold, 'clip_image')

try:
    os.makedirs(clip_image_path)
    print("Folder 'clip_image' created successfully")
except FileExistsError:
    shutil.rmtree(clip_image_path)
    os.makedirs(clip_image_path)
    print("Folder 'clip_image' already exist")

Folder 'clip_image' created successfully


In [20]:
# Process and save patches
# patch_size = 512

# patch_size = 256

d = 144
patch_size = int(3.5*d)

# patch_size = 140
patches = []
for i, coord in tqdm(enumerate(adata.obsm['spatial']), total=len(adata.obsm['spatial'])):
    # print(coord)
    # Calculate patch coordinates
    left = int(coord[0] - patch_size / 2)
    top = int(coord[1] - patch_size / 2)
    right = left + patch_size
    bottom = top + patch_size

    # Extract patch
    patch = im[top:bottom, left:right]

    # Resize patch to 512x512, using INTER_LINEAR for both upsizing and downsizing
    if patch_size != 512:
        resized_patch = cv2.resize(patch, (512, 512), interpolation=cv2.INTER_LINEAR)
    else:
        resized_patch = patch
    # Save resized patch
    cv2.imwrite(os.path.join(clip_image_path, f'{i}.png'), resized_patch)

100%|███████████████████████████████████████| 3673/3673 [00:43<00:00, 85.36it/s]


<h4>filtering</h4>

In [22]:
def process_image(filename, path, output_path, GaussianBlur, lower, upper):
    image_path = os.path.join(path, filename)
    image = cv2.imread(image_path, 0)
    original_height, original_width = image.shape[:2]

    resize_needed = (original_height != 512 or original_width != 512)
    if resize_needed:
        image = cv2.resize(image, (512, 512))

    if GaussianBlur:
        image = cv2.GaussianBlur(image, (5, 5), 0)
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    image_shape = image.shape
    custom_mask = create_custom_mask(image_shape, lower, lower, upper, upper)
    fshift_masked = fshift * custom_mask
    f_ishift = np.fft.ifftshift(fshift_masked)
    image_filtered = np.fft.ifft2(f_ishift)
    image_filtered = np.abs(image_filtered)
    if GaussianBlur:
        image_filtered = cv2.GaussianBlur(image_filtered, (15, 15), 0)
    image_filtered_rgb = cv2.cvtColor(np.float32(image_filtered), cv2.COLOR_GRAY2RGB)
    mae_patch = cv2.resize(image_filtered_rgb, (224, 224), interpolation=cv2.INTER_LINEAR)

    if resize_needed:
        image_filtered_rgb = cv2.resize(image_filtered_rgb, (original_width, original_height))
    cv2.imwrite(os.path.join(output_path, filename), image_filtered_rgb)
    return mae_patch

def create_custom_mask(image_shape, x1, y1, x2, y2):
    rows, cols = image_shape
    mask = np.zeros((rows, cols), np.uint8)
    mask[y1:y2, x1:x2] = 1
    return mask


path = os.path.join(file_fold ,'clip_image')
output_path =  os.path.join(file_fold,'clip_image_filter')
GaussianBlur = True
upper = 275
lower = 245

try:
    os.makedirs(output_path)
    print("Folder 'clip_image_filter' created successfully")
except FileExistsError:
    shutil.rmtree(output_path)
    os.makedirs(output_path)
    print("Folder 'clip_image_filter' already exist")

png_files = [name for name in os.listdir(path) if name.endswith('.png')]
args = [(filename, path, output_path, GaussianBlur, lower, upper) for filename in png_files]

with Pool(processes=os.cpu_count() - 2) as pool:
    patches = pool.starmap(process_image, args)

Folder 'clip_image_filter' already exist


[ WARN:0@707.705] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.705] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.720] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.720] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.720] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.723] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.728] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.728] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@707.732] global loadsav

<h4>BYOL</h4>

In [23]:
def process_images(epoch_num):
    path = os.path.join(file_fold,'clip_image_filter')

    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
        torch.cuda.manual_seed_all(42)
    random.seed(42)
    np.random.seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' 
    torch.use_deterministic_algorithms(True)

    class CustomDataset(Dataset):
        def __init__(self, root_dir, transform=None):
            self.root_dir = root_dir
            self.transform = transform
            self.image_list = sorted([x for x in os.listdir(root_dir) if x.endswith('.png')], key=lambda x: int(x.split('.')[0]))

        def __len__(self):
            return len(self.image_list)

        def __getitem__(self, idx):
            img_name = os.path.join(self.root_dir, self.image_list[idx])
            img = Image.open(img_name).convert('RGB')

            if self.transform:
                img = self.transform(img)

            return img

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    dataset = CustomDataset(path, transform=transform)
    data_loader = DataLoader(dataset, batch_size=21, shuffle=True, num_workers=1) 

    class RandomApply(nn.Module):
        def __init__(self, fn, p):
            super().__init__()
            self.fn = fn
            self.p = p

        def forward(self, x):
            if random.random() > self.p:
                return x
            return self.fn(x)

    DEFAULT_AUG = torch.nn.Sequential(
        RandomApply(T.ColorJitter(0.8, 0.8, 0.8, 0.2), p=0.3),
        T.RandomGrayscale(p=0.2),
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip(),
        RandomApply(T.GaussianBlur((3, 3), (1.0, 2.0)), p=0.2),
        T.RandomRotation(degrees=(0, 360)),
        T.RandomResizedCrop((256, 256)),
        T.Normalize(mean=torch.tensor([0.485, 0.456, 0.406]), std=torch.tensor([0.229, 0.224, 0.225])),
    )

    learner = BYOL(
            models.resnet50(pretrained=True),
            image_size=256,
            hidden_layer='avgpool',
            augment_fn=DEFAULT_AUG
        )
    if torch.cuda.is_available():
        learner = learner.cuda()

    opt = torch.optim.Adam(learner.parameters(), lr=3e-4)

    print('start training!')
    for epoch in range(epoch_num):
        start_time = time.time()
        for images in data_loader:
            images = images.cuda() if torch.cuda.is_available() else images
            loss = learner(images)
            opt.zero_grad()
            loss.backward()
            opt.step()
            learner.update_moving_average()
        end_time = time.time()
        print(f'Epoch [{epoch + 1}/{epoch_num}], Loss: {loss.item():.4f}, Time: {end_time - start_time:.2f} seconds')

    torch.save(learner.state_dict(), 'learner.pth')

    learner.eval()
    embeddings = []
    print('start eval!')
    for i in tqdm(range(len(dataset)), desc='Evaluation Progress'):
        img = dataset[i]
        img = img.cuda() if torch.cuda.is_available() else img
        with torch.no_grad():
            _, embedding = learner(img.unsqueeze(0), return_embedding=True)
            embeddings.append(embedding.cpu().numpy())

    embeddings = np.vstack(embeddings)
    np.save(os.path.join(file_fold,'embeddings.npy'), embeddings)
    print(embeddings.shape)

    return embeddings

epoch_num=3
embeddings = process_images(epoch_num)

/home/isaac/dev/sfu/cmpt415/CLGraph/CLGraph/clgraph/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/isaac/dev/sfu/cmpt415/CLGraph/CLGraph/clgraph/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/isaac/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|███████████████████████████████████████| 97.8M/97.8M [00:00<00:00, 112MB/s]


start training!
Epoch [1/3], Loss: 0.1949, Time: 60.40 seconds
Epoch [2/3], Loss: 0.1238, Time: 59.00 seconds
Epoch [3/3], Loss: 0.3096, Time: 59.20 seconds
start eval!


Evaluation Progress: 100%|█████████████████| 3673/3673 [00:32<00:00, 112.28it/s]


(3673, 2048)


In [26]:
np.save(osp.join(file_fold,"embeddings.npy"),embeddings)